In [1]:
import pandas as pd
from config import gkey, yelp_key
from pprint import pprint
import requests
import string

In [2]:
farm_markets = pd.read_csv("../farmers_markets.csv")

farm_markets.columns

Index(['FMID', 'MarketName', 'Website', 'Facebook', 'Twitter', 'Youtube',
       'OtherMedia', 'street', 'city', 'County', 'State', 'zip', 'Season1Date',
       'Season1Time', 'Season2Date', 'Season2Time', 'Season3Date',
       'Season3Time', 'Season4Date', 'Season4Time', 'x', 'y', 'Location',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP', 'Organic', 'Bakedgoods',
       'Cheese', 'Crafts', 'Flowers', 'Eggs', 'Seafood', 'Herbs', 'Vegetables',
       'Honey', 'Jams', 'Maple', 'Meat', 'Nursery', 'Nuts', 'Plants',
       'Poultry', 'Prepared', 'Soap', 'Trees', 'Wine', 'Coffee', 'Beans',
       'Fruits', 'Grains', 'Juices', 'Mushrooms', 'PetFood', 'Tofu',
       'WildHarvested', 'updateTime'],
      dtype='object')

In [3]:
farm_markets.head()

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,...,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,...,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,...,N,N,N,N,N,N,N,N,N,2013
3,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,...,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM
4,1002454,112st Madison Avenue,NaN,NaN,NaN,NaN,NaN,112th Madison Avenue,New York,New York,...,N,N,N,N,N,N,N,N,N,3/1/2012 10:38:22 AM


In [4]:
# Drop the following columns to make a "clean" dataset

farm_markets_dropped = farm_markets.drop(["FMID", "Website", "Facebook", "Twitter", "Youtube",
                                       "OtherMedia", "street", "Location"], axis=1)
farm_markets_dropped.shape

(8730, 51)

In [5]:
farm_markets_newcols = farm_markets_dropped.rename(columns={"x":"Longitude",
                                                       "y":"Latitude"})
farm_markets_newcols.head()

,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,Caledonia Farmers Market Association - Danville,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,...,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,Stearns Homestead Farmers' Market,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,106 S. Main Street Farmers Market,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,2013
3,10th Steet Community Farmers Market,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM
4,112st Madison Avenue,New York,New York,New York,10029,July to November,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,3/1/2012 10:38:22 AM


In [6]:
# Drop null lat/long: 29 farmers markets

farm_markets_validcoords = farm_markets_newcols.dropna(subset=['Latitude', 'Longitude'])

farm_markets_validcoords["zip"].count()

7791

In [7]:
# Set up URL to make API calls
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
params = {"key":gkey}

# Make copy so as not to overwrite data
farm_markets_clean = farm_markets_validcoords.copy()

# Replace NaN zipcodes or zipcodes with too few digits
for index, row in farm_markets_validcoords.iterrows():
    if pd.isnull(row["zip"]) or (len(row["zip"]) != 5) or (row["zip"].isdigit() == False):
        params["latlng"] = f'{row["Latitude"]},{row["Longitude"]}'
        response = requests.get(base_url, params=params).json()
        
        # Find the response entry with zip codes, then add zip code to dataframe
        for info_dict in response["results"][0]["address_components"]:
            if "postal_code" in info_dict["types"]:
                farm_markets_clean.loc[index, "zip"] = info_dict["long_name"]

In [8]:
farm_markets_clean["zip"]

0       05828
1       44129
2       29682
3       64759
4       10029
5       37204
6       10027
7       19801
8       20009
9       20011
10      97212
11      10452
12      10033
13      55413
14      23219
15      19146
16      69361
17      61920
18      32626
19      98072
20      66604
21      19146
22      07732
23      84341
24      19129
25      19146
26      24521
27      45402
28      60450
29      32461
        ...  
8699    49504
8701    28801
8702    23462
8703    43050
8704    92116
8705    29745
8706    68467
8707    03909
8708    03909
8709    10598
8710    47396
8711    23690
8712    52315
8713    94599
8714    78626
8715    33150
8716    33054
8717    96097
8718    95991
8719    95991
8720    92399
8721    57718
8722    43701
8723    49464
8724    19525
8725    87004
8726    55398
8727    84767
8728    46077
8729    64153
Name: zip, Length: 8701, dtype: object

In [74]:
# Make Yelp API calls

search_url = "https://api.yelp.com/v3/businesses/search?"
headers = {"Authorization":"Bearer " + yelp_key}

# Initialize new columns
farm_markets_clean["Yelp Results"] = ""
farm_markets_clean["Yelp Price"] = ""
farm_markets_clean["Yelp Rating"] = ""
farm_markets_clean["Yelp ID"] = ""

params = {
    "radius":2000,
    "categories":"farmersmarket",
    "limit":5
}

# Yelp Business Search: obtain business ID, price level, and rating
# NOTE: Limit of 5000 requests per day. Day 1, indices 0-4999, Day 2, 5000+
for index, row in farm_markets_clean.iterrows():
    if index < 499:
        print(index)
        print(row["MarketName"])
        
        params["term"] = row["MarketName"]
        
        # Could search by latitude/longitude or zip code
        # num farmers markets returned from search of 500 with lat/long: 82
        # num farmers markets returned from search of 500 with zip: 55
        # --> search with lat/long
        params["latitude"] = row["Latitude"]
        params["longitude"] = row["Longitude"]

        response = requests.get(search_url, params=params, headers=headers).json()

        try:
            result = response["businesses"][0]
            print(result["name"])
            farm_markets_clean.loc[index, "Yelp Results"] = response["businesses"]
            farm_markets_clean.loc[index, "Yelp Price"] = result["price"]
            farm_markets_clean.loc[index, "Yelp Rating"] = result["rating"]
        except:
            print(f"No nearby market for {row['MarketName']}")

0
 Caledonia Farmers Market Association - Danville
No nearby market for  Caledonia Farmers Market Association - Danville
1
 Stearns Homestead Farmers' Market
No nearby market for  Stearns Homestead Farmers' Market
2
106 S. Main Street Farmers Market
No nearby market for 106 S. Main Street Farmers Market
3
10th Steet Community Farmers Market
No nearby market for 10th Steet Community Farmers Market
4
112st Madison Avenue
No nearby market for 112st Madison Avenue
5
12 South Farmers Market
12 South Farmers Market
6
125th Street Fresh Connect Farmers' Market
No nearby market for 125th Street Fresh Connect Farmers' Market
7
12th & Brandywine Urban Farm Market
No nearby market for 12th & Brandywine Urban Farm Market
8
14&U Farmers' Market
No nearby market for 14&U Farmers' Market
9
14th & Kennedy Street Farmers Market
14th & Kennedy St Farmers Market
10
16th Ave Farmers Market
No nearby market for 16th Ave Farmers Market
11
170 Farm Stand
No nearby market for 170 Farm Stand
12
175th Street Gr

No nearby market for Airport Farmers and Flea Market
103
Akebono Farmers’ Market
No nearby market for Akebono Farmers’ Market
104
Akers of Strawberries
Akers of Strawberries
105
Akron Farmers Market
No nearby market for Akron Farmers Market
106
Akwesasne Farmers Market
No nearby market for Akwesasne Farmers Market
107
Alabama Gulf Coast Market
No nearby market for Alabama Gulf Coast Market
108
Alachua County Farmers' Market
No nearby market for Alachua County Farmers' Market
109
Alamance County Farmers Market
No nearby market for Alamance County Farmers Market
110
Alameda Farmers' Market
Alameda Farmers' Market
No nearby market for Alameda Farmers' Market
111
Alamo Farmers Market
No nearby market for Alamo Farmers Market
112
Alamo Heights Farmers Market
No nearby market for Alamo Heights Farmers Market
113
Alamogordo Farmers' Market in Alameda Park
No nearby market for Alamogordo Farmers' Market in Alameda Park
114
Alamogordo Otero County Farmers' Market
No nearby market for Alamogordo

Andersonville Farmers Market
No nearby market for Andersonville Farmers Market
204
Andover Farmers Market
No nearby market for Andover Farmers Market
205
Andover Farmers' Market
No nearby market for Andover Farmers' Market
206
Andrews Farmers Market
No nearby market for Andrews Farmers Market
207
Angelica Farmers' Market
Catbird Griddle
208
Angelina County Farmers Market
Angelina County Farmer's Market
209
Angels Camp Farmers Market-Fresh Fridays
Angels Camp Farmers Market
210
Ann Arbor Farmers Market
Ann Arbor Farmers Market
No nearby market for Ann Arbor Farmers Market
211
Anna House Farm Stand
No nearby market for Anna House Farm Stand
212
Anna/Union County Farmers Market
No nearby market for Anna/Union County Farmers Market
213
Annandale Farmers Market 
Annandale Farmer's Market
214
Annandale Farmers Market - ACT
No nearby market for Annandale Farmers Market - ACT
215
Anne Arundel County Farmers' Market  (Riva)
No nearby market for Anne Arundel County Farmers' Market  (Riva)
216
An

Ashe County Farmers' Market
294
Asheville City Market
No nearby market for Asheville City Market
295
Asheville City Market - South
No nearby market for Asheville City Market - South
296
Ashfield Farmers Market
No nearby market for Ashfield Farmers Market
297
ASHFORD FARMERS’ MARKET
No nearby market for ASHFORD FARMERS’ MARKET
298
Ashland Area Farmers Market
No nearby market for Ashland Area Farmers Market
299
Ashland Farmers Market
Ashland Farmers Market
No nearby market for Ashland Farmers Market
300
Ashland Farmers Market
Ashland Farmers' Market
301
Ashland Tuesday Market
Rogue Valley Growers & Crafters Market
302
Ashland United Methodist Church Farmers Market
No nearby market for Ashland United Methodist Church Farmers Market
303
Ashley County Farmers Market
No nearby market for Ashley County Farmers Market
304
Ashley Farmers Market
No nearby market for Ashley Farmers Market
305
Ashley Valley Farmers' Market (Vernal)
No nearby market for Ashley Valley Farmers' Market (Vernal)
306
As

No nearby market for Baldwin Park Farmers Market
397
Baldwin's Main St. Farmers Market
No nearby market for Baldwin's Main St. Farmers Market
398
Ballard County Farmers' Market
No nearby market for Ballard County Farmers' Market
399
Ballard Farmers Market
Ballard Sunday Farmers Market
No nearby market for Ballard Farmers Market
400
Ballston Farmers Market
Ballston Farmers Market
401
Ballston Spa Farmers Market
No nearby market for Ballston Spa Farmers Market
402
Balsam Lake Farmers Market
No nearby market for Balsam Lake Farmers Market
403
Baltimore County Farmers' Market
No nearby market for Baltimore County Farmers' Market
404
Baltimore Farmers' Market
Baltimore Farmers' Market & Bazaar
No nearby market for Baltimore Farmers' Market
405
Bancroft Street Farmers Market
No nearby market for Bancroft Street Farmers Market
406
Bandera Farmers Market
No nearby market for Bandera Farmers Market
407
Bangor Farmers Market
No nearby market for Bangor Farmers Market
408
Bangor Farmers' Market
B

In [82]:
valid_yelp = farm_markets_clean[farm_markets_clean["Yelp Results"] != ""]
valid_yelp = valid_yelp.reset_index(drop=True)

valid_yelp.shape

# Verify that market from Yelp matches market from CSV file
for index, row in valid_yelp.iterrows():
    
    # convert to lowercase and strip apostrophes
    csv_name = row["MarketName"].lower().replace("'","")
    yelp_name = row["Yelp Results"]["name"].lower().replace("'","")
    print(csv_name, yelp_name)
    print("after strip")
    
    csv_name = csv_name.replace("farmers market", "")
    yelp_name = yelp_name.replace("farmers market", "")
    print(csv_name, yelp_name)
    
    csv_words = csv_name.split(" ")
    print(csv_words)
    yelp_words = yelp_name.split(" ")
    print(yelp_words)
    
    overlap_words = False
    
    for word in csv_words:
        if (word.isalnum()) and (word in yelp_words):
            overlap_words = True
    
    if not((csv_name == yelp_name) or overlap_words):
        row["Yelp Results"] = "NOT EQUAL"
    
valid_yelp[valid_yelp["Yelp Results"] == "NOT EQUAL"]
# valid_yelp = valid_yelp[valid_yelp["Yelp Results"] != "NOT EQUAL"]

#     try:
#         print(row["Yelp Results"]["name"])
#     except:
#         print(row["Yelp Results"])
        
# strip apostrophes
# convert to lowercase
# remove "farmers market"
# test if new strings are equal OR if string1 contains word in string2 (for loop)

12 south farmers market 12 south farmers market
after strip
12 south  12 south 
['12', 'south', '']
['12', 'south', '']
14th & kennedy street farmers market 14th & kennedy st farmers market
after strip
14th & kennedy street  14th & kennedy st 
['14th', '&', 'kennedy', 'street', '']
['14th', '&', 'kennedy', 'st', '']
175th street greenmarket 175th street greenmarket
after strip
175th street greenmarket 175th street greenmarket
['175th', 'street', 'greenmarket']
['175th', 'street', 'greenmarket']
21 acres farm market 21 acres
after strip
21 acres farm market 21 acres
['21', 'acres', 'farm', 'market']
['21', 'acres']
3 french hens french country market 3 french hens french country market
after strip
3 french hens french country market 3 french hens french country market
['3', 'french', 'hens', 'french', 'country', 'market']
['3', 'french', 'hens', 'french', 'country', 'market']
38th & meridian farmers market  38th & meridian street farmers market
after strip
38th & meridian   38th & merid

,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,...,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime,Yelp ID,Yelp Price,Yelp Rating,Yelp Results


False